## Estimation: Bias and Verification of Standard Errors

Methods/Functions

`mean`, `var`, `RandomState`, `RandomState.chisquare`, `array`, `DataFrame.plot.kde`, `stats.norm.ppf`

### Exercise 25
Simulate a set of i.i.d. $\chi_{5}^{2}$ random variables and use the method of moments
to estimate the mean and variance.

In [127]:
import numpy as np

rs = np.random.RandomState(19991231)

rvs = rs.chisquare(5, size = 10)

mu_rvs = rvs.mean()
var_rvs = rvs.var()

print(f"The mean is {mu_rvs} and the variance is {var_rvs}")

The mean is 5.018686987866649 and the variance is 7.076920224073575


### Exercise 26
Compute the asymptotic variance of the method of moment estimator.

In [128]:
err = rvs - mu_rvs
mom_err = np.array([err, err**2 - var_rvs]).T
nobs = mom_err.shape[0]

acov = mom_err.T @ mom_err / nobs
print(f"The asymptomatic covariance is {acov}")


The asymptomatic covariance is [[ 7.07692022 -0.14669921]
 [-0.14669921 37.06172824]]


### Exercise 27
Repeat Exercises 24 and 25 a total of 1000 times.
Examine the finite sample bias of these estimators relative to the true values.

In [129]:
estimates = np.zeros((1000,2))
acov = np.zeros((1000,2,2))
for i in range(1000):
    rvs = rs.chisquare(5,size=10)
    rvs_mu = rvs.mean()
    rvs_var = rvs.var()
    estimates[i] = [rvs_mu, rvs_var]


    errs = rvs - rvs_mu
    mom_errs = np.array([errs,errs**2 - rvs_var]).T
    nobs = mom_errs.shape[0]

    acov[i] = mom_errs.T @ mom_errs / nobs

bias = estimates.mean(axis=0) - np.array([5,10])
print(f"The bias of the mean is {bias[0]} and the bias of the variance is {bias[1]}")

The bias of the mean is 0.030637114579928237 and the bias of the variance is -0.6459309147194912


### Exercise 28
Repeat Exercises 24 and 25 a total of 1000 times.
Compare the covariance of the estimated means and variance (1000 of each) to the asymptotic covariance of the parameters (use the average of the 1000 estimated variance-covariances). Are these close? How does the sample size affect this?

In [130]:
est_errs = estimates - estimates.mean(axis=0)
finite_sample_cov = est_errs.T @ est_errs / 1000
avg_acov = acov.mean(0)

print("The finite sample covariance is:")
print(finite_sample_cov)

print("The average asymptotic covariance of the parameters is:")
print(avg_acov/10)

The finite sample covariance is:
[[ 1.11324227  4.20444024]
 [ 4.20444024 39.84465645]]
The average asymptotic covariance of the parameters is:
[[ 0.93540691  3.04327717]
 [ 3.04327717 29.41246286]]


### Exercise 29
In the previous problem, for each parameter, form a standardized parameter estimate as

$$z_{i}=\frac{\sqrt{n}\left(\hat{\theta}_{i}-\theta_{i,0}\right)}{\sqrt{\hat{\Sigma}_{ii}}}$$ 

where

$$\sqrt{n}\left(\hat{\theta}-\theta_{0}\right)\stackrel{d}{\rightarrow} N\left(0,\Sigma\right)$$

so that $\hat{\Sigma}$ is the estimated asymptotic covariance. What percent of these $z_{i}$
are larger in absolute value than 10%, 5% and 1% 2-sided critical values from a normal?

In [ ]:
from scipy import stats
import pandas as pd





### Exercise 30
Produce a density plot of the $z_{i}$ standardized parameters and compare to a standard normal.

### Exercise 31
Repeat the same exercise for the Bernoulli problem from the previous question.